### purpose

validate climate outlier scenarios for the 6-trait/multivariate sim (kicked off in 02_analysis/04_outlier_climate_runs/07_submit_multivariate_jobs)

### notes

I developed this code using GF outputs jand double checked that I was calculating the same validation score for 6-trait climate outlier GF as was calculated with MVP_15.py. I do not overwrite the previous GF validation file output (GF is commented out on final run)

In [1]:
from pythonimports import *

import MVP_summary_functions as mvp

t0 = dt.now()  # notebook timer

outlier_outerdir = '/home/b.lind/offsets/climate_outlier_runs/tutorial/run_20220919_tutorial'
fitness_dir =      '/home/b.lind/offsets/climate_outlier_runs/tutorial/fitness_mats'

method_dirs = {
    'lfmm' : f'{outlier_outerdir}/lfmm2/lfmm_outiles',  # outfiles was spelled incorrectly, doh!
    'rda' : f'{outlier_outerdir}/rda/offset_outfiles'
#     'GF' : f'{outlier_outerdir}/GF/fitting_outfiles'
}

mvp.latest_commit()
session_info.show()

#########################################################
Today:	September 07, 2023 - 11:57:56 EDT
python version: 3.8.5
conda env: mvp_env

Current commit of pythonimports:
commit 9493334506a1f2926fba0734fa74ba2f87194611  
Author: Brandon Lind <lind.brandon.m@gmail.com>  
Date:   Fri Jun 30 09:41:00 2023 -0400

Current commit of MVP_offsets:
commit 34fec27ce185eb670eb16124a5255c9d4964ee70  
Author: Brandon Lind <lind.brandon.m@gmail.com>  
Date:   Tue Sep 5 11:47:40 2023 -0400
#########################################################



# read in offset dfs

In [2]:
offset_dfs = wrap_defaultdict(list, 3)
for program, d in method_dirs.items():  # loop works for lfmm and GF (GF was done using MVP_15.py)
    if program == 'rda':
        continue
    
    files = fs(d, endswith='.txt')
    
    dfs = wrap_defaultdict(list, 2)
    for f in pbar(files, desc=program):
        if program == 'GF':
            *_, marker_set, scenario = op.basename(f).rstrip('_gradient_forest_offset.txt').split("_")
        else:
            assert program == 'lfmm'
            *_, marker_set, scenario = op.basename(f).rstrip('.txt').split("_")
        
        df = pd.read_table(f).T
        df.index = [scenario]

        dfs[program][marker_set].append(df)
    
    for program in dfs:
        for marker_set, df_list in dfs[program].items():
            df = pd.concat(df_list)
            df.index = df.index.astype(float)
            offset_dfs[program][marker_set] = df
            assert df.shape == (12, 100)
            
# now handle rda
files = fs(method_dirs['rda'], endswith='.txt')
for f in pbar(files, desc='rda'):
    *_, marker_set, ntraits6, structcorr = op.basename(f).split('_rda_offset.txt')[0].split("_")
    
    df = pd.read_table(f, delim_whitespace=True)
    df.columns = df.columns.astype(int)
    df.index = df.index.astype(float)
    
    assert df.shape == (12, 100)
    
    offset_dfs['rda'][marker_set][structcorr] = df

            
offset_dfs['lfmm']['adaptive']

rda: 100%|███████████████| 8/8 [00:00<00:00, 117.26it/s]


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0.00,0.658088,0.580224,0.400947,0.411471,0.445760,0.512149,0.433955,0.598798,0.206644,0.283786,0.504413,0.415154,0.463752,0.646513,0.305306,0.448262,0.420454,0.278157,0.205412,0.299929,0.701758,0.558889,0.494661,0.527242,0.372422,0.516490,0.503227,0.416065,0.134411,0.468967,0.448789,0.318245,0.545371,0.273762,0.326767,0.435728,0.376649,0.344447,0.302062,0.442166,0.479524,0.469797,0.354545,0.494690,0.450461,0.428190,0.380298,0.153145,0.356122,0.508439,0.702202,0.622108,0.612062,0.487136,0.491403,0.433594,0.153630,0.183501,0.410170,0.572035,0.596710,0.513234,0.350702,0.351113,0.397313,0.341675,0.272255,0.543678,0.717531,0.624026,0.580642,0.600042,0.412657,0.411319,0.470906,0.214232,0.569671,0.684823,0.505388,0.682701,0.542518,0.475050,0.568116,0.546601,0.415055,0.286449,0.489550,0.630670,0.614535,0.753710,0.709172,0.376909,0.588235,0.357529,0.486534,0.472760,0.525276,0.843159,0.720631,0.865702
1.72,0.805754,0.790897,0.647495,0.592764,0.654594,0.836444,0.666307,0.885001,0.608783,0.664399,0.771915,0.723997,0.716737,0.745837,0.589608,0.766727,0.697211,0.594142,0.593291,0.693584,0.878735,0.790819,0.688090,0.651410,0.706339,0.821020,0.734453,0.707859,0.572926,0.743745,0.789101,0.634992,0.670068,0.616183,0.699999,0.745897,0.683174,0.721068,0.661264,0.725492,0.713580,0.695275,0.641834,0.849001,0.838198,0.751482,0.705209,0.603377,0.717315,0.776414,0.874024,0.842867,0.761402,0.821239,0.821388,0.807084,0.589632,0.598276,0.712174,0.822496,0.826764,0.761481,0.696065,0.731910,0.811585,0.759359,0.642071,0.826790,0.876963,0.889862,0.809769,0.715863,0.467233,0.749265,0.843756,0.615117,0.834167,0.849156,0.804592,0.903971,0.771241,0.526511,0.676700,0.855589,0.766084,0.657140,0.785548,0.860327,0.861588,0.912148,0.830435,0.495188,0.657937,0.664865,0.686982,0.768942,0.806943,0.952886,0.871158,0.944994
2.35,0.835576,0.823819,0.703670,0.654476,0.715950,0.868110,0.737510,0.899943,0.679066,0.745914,0.806757,0.767074,0.760062,0.794908,0.655886,0.814709,0.741911,0.660612,0.661147,0.767323,0.908927,0.839153,0.749651,0.702865,0.766608,0.850475,0.775434,0.758921,0.640027,0.788683,0.845487,0.695927,0.732028,0.682202,0.772697,0.788327,0.726666,0.779776,0.707054,0.776235,0.774582,0.755584,0.718678,0.881869,0.886153,0.791703,0.749410,0.679969,0.760830,0.830568,0.881360,0.859460,0.820479,0.871952,0.871165,0.851801,0.659068,0.665876,0.750020,0.860177,0.870012,0.809123,0.784866,0.809251,0.863424,0.813044,0.701891,0.855888,0.898281,0.910533,0.870767,0.790155,0.628224,0.820534,0.893151,0.679726,0.854138,0.856209,0.833207,0.927166,0.852390,0.669683,0.806360,0.911719,0.824608,0.726045,0.805650,0.885137,0.889634,0.932744,0.872996,0.641638,0.784404,0.752345,0.753185,0.817546,0.846337,0.974302,0.889603,0.968278
2.74,0.844082,0.836952,0.731129,0.683567,0.743534,0.880488,0.765723,0.906899,0.711829,0.781895,0.820696,0.786039,0.784018,0.807746,0.687049,0.843106,0.763218,0.693048,0.694079,0.804420,0.927000,0.862237,0.773438,0.724938,0.800609,0.864996,0.792827,0.784065,0.673976,0.814354,0.870823,0.727088,0.752325,0.711749,0.804560,0.809484,0.749046,0.806458,0.732880,0.801772,0.806972,0.780585,0.750163,0.895178,0.903803,0.810728,0.771908,0.716573,0.780188,0.853999,0.912169,0.856040,0.839365,0.891285,0.889592,0.869527,0.691847,0.698812,0.770215,0.880385,0.908277,0.828821,0.817982,0.837230,0.883438,0.837015,0.729094,0.863328,0.907424,0.909177,0.909880,0.832817,0.692559,0.848201,0.911668,0.711810,0.863882,0.871422,0.846798,0.928469,0.891565,0.732900,0.848510,0.928295,0.852369,0.759150,0.827204,0.894385,0.899160,0.941786,0.928072,0.707450,0.834750,0.794455,0.785090,0.845745,0.864690,0.974553,0.897983,0.978409
3.13,0.849076,0.844797,0.754317,0.709427,0.765471,0.889573,0.786182,0.918200,0.738597,0

# read in fitness dfs

In [3]:
files = fs(fitness_dir, endswith='.txt')

assert len(files) == 12  # number of climate outlier scenarios

dfs = []
for f in files:
    df = pd.read_table(f)
    
    scenario = op.basename(f).rstrip('.txt').split('_')[-1]
    df.index = [scenario]
    
    dfs.append(df)
    
fitness_mat = pd.concat(dfs)
fitness_mat.columns = fitness_mat.columns.astype(int)
fitness_mat.index = fitness_mat.index.astype(float)

fitness_mat

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0.00,0.738889,0.786154,0.790,0.737,0.810000,0.930833,0.762222,0.772222,0.862000,0.921,0.810000,0.852222,0.81000,0.73625,0.823333,0.886667,0.818182,0.834,0.888182,0.937273,0.9275,0.867500,0.738571,0.683077,0.845833,0.880000,0.77125,0.860000,0.953077,0.926364,0.86125,0.825000,0.755,0.880000,0.930909,0.825714,0.760833,0.90625,0.918,0.941111,0.79750,0.797,0.815556,0.9275,0.927500,0.868571,0.826,0.914667,0.876,0.915714,0.77625,0.798,0.883636,0.944615,0.942727,0.93125,0.888571,0.874286,0.853846,0.882222,0.848462,0.850000,0.89375,0.898182,0.92125,0.927895,0.851,0.81125,0.87000,0.8075,0.87750,0.900000,0.880667,0.870000,0.930000,0.932727,0.855455,0.848571,0.828889,0.790000,0.887143,0.902,0.842,0.879231,0.937,0.924545,0.870,0.853333,0.774667,0.730,0.906667,0.883158,0.843333,0.900000,0.927778,0.909,0.897143,0.836154,0.750000,0.734615
1.72,0.681111,0.622308,0.460,0.356,0.447143,0.581667,0.731111,0.672222,0.603333,0.668,0.584286,0.500000,0.37625,0.38125,0.462222,0.684167,0.720909,0.627,0.523636,0.620000,0.4925,0.436667,0.382857,0.412308,0.547500,0.709167,0.71500,0.576667,0.575385,0.563636,0.38375,0.402857,0.421,0.486667,0.626364,0.741429,0.638333,0.60125,0.563,0.597778,0.37625,0.460,0.514444,0.5575,0.649167,0.714286,0.639,0.599333,0.530,0.580000,0.41125,0.492,0.642727,0.650000,0.700909,0.68250,0.612857,0.548571,0.520000,0.580000,0.477692,0.582857,0.78375,0.732727,0.67000,0.616316,0.519,0.47500,0.59875,0.5600,0.51875,0.665333,0.785333,0.696667,0.621250,0.588182,0.515455,0.537143,0.583889,0.520000,0.560000,0.712,0.752,0.647692,0.624,0.590000,0.520,0.607778,0.589333,0.510,0.614444,0.758947,0.726667,0.670000,0.640000,0.603,0.614286,0.678462,0.571429,0.523846
2.35,0.536667,0.466923,0.316,0.224,0.292857,0.403333,0.588889,0.524444,0.436000,0.489,0.427143,0.338889,0.22875,0.24625,0.308889,0.510000,0.561818,0.464,0.354545,0.435455,0.3200,0.280000,0.245714,0.278462,0.386667,0.535000,0.57125,0.411111,0.389231,0.384545,0.23250,0.253571,0.272,0.316667,0.444545,0.584286,0.494167,0.42250,0.382,0.414444,0.23500,0.306,0.355556,0.3800,0.467500,0.548571,0.478,0.421333,0.360,0.402857,0.26500,0.339,0.470909,0.463846,0.515455,0.50000,0.441429,0.378571,0.355385,0.406667,0.317692,0.415714,0.61125,0.557273,0.48375,0.436842,0.356,0.31875,0.42875,0.4000,0.35000,0.486667,0.614667,0.526667,0.439375,0.409091,0.350909,0.372857,0.421111,0.364444,0.385714,0.534,0.589,0.476923,0.442,0.411818,0.354,0.440000,0.434667,0.366,0.433333,0.590000,0.563333,0.493333,0.455556,0.424,0.437143,0.516923,0.424286,0.380000
2.74,0.442222,0.373846,0.238,0.158,0.215714,0.302500,0.487778,0.425556,0.336000,0.380,0.334286,0.252222,0.16125,0.17625,0.226667,0.404167,0.458182,0.366,0.264545,0.333636,0.2300,0.201667,0.177143,0.206923,0.294167,0.429167,0.47000,0.315556,0.290769,0.290909,0.16375,0.180000,0.201,0.235556,0.340909,0.478571,0.400000,0.32250,0.285,0.311111,0.16500,0.224,0.267778,0.2850,0.361667,0.438571,0.377,0.319333,0.269,0.304286,0.19250,0.252,0.363636,0.356154,0.405455,0.39250,0.341429,0.284286,0.265385,0.307778,0.233846,0.320000,0.49875,0.445455,0.38125,0.334737,0.266,0.23625,0.32875,0.3075,0.26000,0.381333,0.500667,0.417778,0.335000,0.311818,0.263636,0.281429,0.324444,0.275556,0.295714,0.424,0.480,0.373077,0.337,0.310000,0.265,0.341111,0.343333,0.282,0.332222,0.475789,0.456667,0.385556,0.350000,0.324,0.335714,0.409231,0.331429,0.295385
3.13,0.351111,0.285385,0.170,0.108,0.152857,0.219167,0.387778,0.333333,0.251333,0.284,0.250000,0.180000,0.11000,0.12250,0.158889,0.308333,0.357273,0.278,0.189091,0.242727,0.1625,0.139167,0.122857,0.148462,0.216667,0.329167,0.37375,0.230000,0.208462,0.206364,0.10875,0.124286,0.140,0.166667,0.252727,0.375714,0.310833,0.23625,0.204,0.225556,0.11125,0.161,0.194444,0.20

# validate

In [4]:
mvp.block_pops

{'northwest': [91, 92, 93, 81, 82, 83, 71, 72, 73],
 'rangecenter': [54, 55, 56, 44, 45, 46, 34, 35, 36],
 'southeast': [28, 29, 30, 18, 19, 20, 8, 9, 10]}

In [5]:
# add all populations as the full set of population names
mvp.block_pops.update({'all' : fitness_mat.columns.tolist()})

In [6]:
mvp.block_pops.keys()

dict_keys(['northwest', 'rangecenter', 'southeast', 'all'])

In [7]:
dfs = defaultdict(list)
for (program, marker_set, *structcorr), offset_df in unwrap_dictionary(offset_dfs):
    
    for block, pops in sorted(mvp.block_pops.items()):
        corrs = offset_df[pops].corrwith(fitness_mat[pops], axis=1, method='kendall')

        corr_df = pd.DataFrame(corrs, columns=['score']).rename_axis(index='outlier_clim').reset_index()
        corr_df['seed'] = 'tutorial'
        corr_df['program'] = program
        corr_df['block'] = block
        corr_df['marker_set'] = marker_set
        
        corr_df = corr_df[['seed', 'marker_set', 'program', 'outlier_clim', 'score', 'block']]
        
        if len(structcorr) > 0:
            corr_df['structcorr'] = structcorr[0]

        dfs[program].append(corr_df)

corr_dfs = {}
for program, df_list in dfs.items():
    corr_dfs[program] = pd.concat(df_list).reset_index(drop=True)
    print(program)
    display(corr_dfs[program].head())


lfmm


,seed,marker_set,program,outlier_clim,score,block
0,tutorial,adaptive,lfmm,0.00,-0.257282,all
1,tutorial,adaptive,lfmm,1.72,-0.028710,all
2,tutorial,adaptive,lfmm,2.35,0.036165,all
3,tutorial,adaptive,lfmm,2.74,0.052329,all
4,tutorial,adaptive,lfmm,3.13,0.059602,all


rda


,seed,marker_set,program,outlier_clim,score,block,structcorr
0,tutorial,CAUSAL,rda,0.00,-0.558253,all,nocorr
1,tutorial,CAUSAL,rda,1.72,-0.395876,all,nocorr
2,tutorial,CAUSAL,rda,2.35,-0.390140,all,nocorr
3,tutorial,CAUSAL,rda,2.74,-0.399838,all,nocorr
4,tutorial,CAUSAL,rda,3.13,-0.403071,all,nocorr


In [8]:
corr_dfs['rda'].marker_set = corr_dfs['rda'].marker_set.map(
    {
        'CAUSAL' : 'adaptive',
        'FALSE' : 'all',
        'NEUTRAL' : 'neutral',
        'TRUE' : 'rda_outliers'
    }
)

corr_dfs['rda'].marker_set.unique()

array(['adaptive', 'all', 'neutral', 'rda_outliers'], dtype=object)

# save

In [9]:
for program, corr_df in corr_dfs.items():
    d = op.dirname(method_dirs[program])

    savedir = makedir(f'{d}/validation')

    f = f'{savedir}/climate_outlier_validation_scores.txt'

    corr_df[corr_df.program == program].to_csv(f, sep='\t', index=False, header=True)

    print(f, '\n')


/home/b.lind/offsets/climate_outlier_runs/tutorial/run_20220919_tutorial/lfmm2/validation/climate_outlier_validation_scores.txt 

/home/b.lind/offsets/climate_outlier_runs/tutorial/run_20220919_tutorial/rda/validation/climate_outlier_validation_scores.txt 



In [10]:
formatclock(dt.now() - t0)

'0-00:00:02'